#Exploring and trimming of fastq data


__How to display at the first 12 lines of a gzipped file?__

The pipe symbol `|` is an extremely powerful function to pass the output from one command directly to the next command. This is often referred to as 'piping'.



In [4]:
!gunzip -c data/A01D_S1_L001_R1_001.fastq.gz | head -n 12

@M02538:27:000000000-ACTEL:1:1101:9229:1157 1:N:0:1
CAGGAACATATAATGGCAAGCCTACGAAAAACGCACCCCCTACTAAAAATCGCTAACAATGCACTAGTCGACCTCCCCGCCCCCTCAAATATTTCAGTAGGATGAAACTTTGGGTCCCTCCTTGGAGTTTTCTTAATTATTCAAAATCCCACTGGGCTTTTCCTTTCAAAACACTACACTTCCGATATTGCTAAAGCTTTTTCTCCCGTCGGACATATCTGCCGAGAAGTAAATTAAGGTTGAATAATTCTAAACCCCCATTCCAATGGGCAATCACTTTTTTTCATCTTGAATCTAATA
+
CCCCCGGGGGFGGGGGGFFGGGGGGGGGGGGGGGGGGEGGGGGFGFGGGGGECEFGGDD<CFGGGGGGGGDGGG,CFG7>FGGG7,:EB<AEFF?<,<,,,55,BFDBEFGF,+@AEG9EFDFGF8,@FF,,7C,A@FE,,C<EE,@>,@A:A:DC,DF?FDFGF,,,@8,@E,6@=DCA83@4+++422@7A,+,,4A8*A0++23*@/***///+++4+1;)0//)21+3++1:)2*+/5))1)9<7?+)9<.(((..,))16:<?:(/*(,.,(.:).)-,.))-)))))))-).))
@M02538:27:000000000-ACTEL:1:1101:16424:1187 1:N:0:1
CAGGAACTAATGGCAAGCCTACGAAAAACCCACCCACTAATAAAAATCGCTAATGACGCACTAGTCGACCTCCCAACACCATCTAACATTTCAACACTATGAAACTTCGGATCCCTCCTAGGATTATGTTTAATTACCCAAATCCTCACGGGATTATTTCTAGCCATACACTACACCTCTGATATCTCCACCGCATTTTCATCAGTAACCCACATCTGCCGAGACGTTAACTACGGCTGACTTATTCGAAACTTACATGCTAATGGAGCATCATTCTTCTTTATCTTCCTT

In [6]:
!zcat -c data/A01D_S1_L001_R2_001.fastq.gz | head -n 12

@M02538:27:000000000-ACTEL:1:1101:9229:1157 2:N:0:1
ATGGAAGGACATAACCAACGAAGGCTGTTATTATGACCAAAAGTAGAAGAATCACTCCAATGTTTCAGGATTCTTTGTATAGGTAAGAGCCATAGTAAAGTCCTCGGCCGATATGCATATAGATACAGATGAAGAAAAAGGATGCGCCATTGGCATGGAGGTTTCGAATTAGTCAACCGTAATTTAAATCTCGGCAGATATGTCCGACGGAGGAAAAAGCTGTAACAATATCGGAAGTGTCGGTTATGGGAAGGAGAAGACCAGTGATAATTGGGAAAATAAAAAAAGAGACAAGGAGGG
+
CCCCC9,@FGGFGFGGFGGGFGGGGFGFGGFGG<EF,,6CCFFGGGA<DGGG,EF<FGGGGGGGGGGGG,@EEFFBFEFFFGG?FFGGGGGGFGGFEFGGGGGGGGGCGGGEEF9FEFF9EFG9DFGGFGGFFGGFEGFGGC=ECDGEE>>;8?DFDCFFD4,,1=DFF=@:EFFBD=D=DAGFFF+9+*C7DC6?CC*;*02<CBAA>A)0:=AF9-*).A8:*/+37*+/->>2).0**/.(,6)))((./0;(,/3(6(((/).)16))*05((((.)).))-,,-..,((((((,(
@M02538:27:000000000-ACTEL:1:1101:16424:1187 2:N:0:1
ATGGAAGTACGTAGCCGACGAAGGCTGTTATTATAACTAGAAGAAATAGGACTACGCCAATATTTCAGGTTTCTTTGTAAAGATATGACCCGTAGTATAGGCCTCGTGCAATATGTATATAAAGGCAGATAAAGAAGAATGATGCTCCATTAGCATGTAAGTTTCGAATAAGTCAGCCGTAGTTAACGTCTCGGCAGATGTGGGTTACTGATGAAAATGCGGTGGAGATATCAGAGGTGTAGTGTATGGCTAGAAATAATCCCGTGAGGGTTTGGGTTATTAAGAATAAAT

Determine the number of sequences in a fastq file

In [7]:
!zcat data/A01D_S1_L001_R1_001.fastq.gz | sed -n '1~4p' | wc -l

39025


In [9]:
!gunzip -c data/A01D_S1_L001_R1_001.fastq.gz | grep " 1"| wc -l

39025


qality trim to a phred score of Q > 30, discarding sequences of length shorter than 90bp.


In [ ]:
%%bash
gunzip data/A01D_S1_L001_R1_001.fastq.gz
fastq_quality_trimmer -Q 33 -t 30 -l 90 -v -i data/A01D_S1_L001_R1_001.fastq -o test1.fastq

Alternatively you could use the pipe function also here:

In [ ]:
!zcat data/A01D_S1_L001_R1_001.fastq.gz | fastq_quality_trimmer -Q 33 -t 30 -l 90 -v -o test1.fastq

Filter all reads with a quality Q<30 in more than 25% of their bases.

In [ ]:
!zcat data/A01D_S1_L001_R2_001.fastq.gz | fastq_quality_filter -Q 33 -q 30 -p 75 -v -o test2.fastq


[Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic) as an example for another efficient software for trimming Illumina data.

To run the below command successfully you will have to specify the correctt input files. Can you determine the desired trimming strategy?

In [ ]:


java -jar /usr/bin/trimmomatic-0.32.jar PE -phred33 -trimlog trimmomatic.log genomic_1.fastq genomic_2.fastq genomic_trimmomatic_paired_1.fastq genomic_trimmomatic_orphan_1.fastq genomic_trimmomatic_paired_2.fastq genomic_trimmomatic_orphan_2.fastq LEADING:30 TRAILING:30 SLIDINGWINDOW:5:20 MINLEN:90


flash for read merging

In [ ]:
flash -h

[vsearch](https://github.com/torognes/vsearch) for read clustering. Note that to perform clustering you'll first have to convert your fastq seqeunces into fasta format.

you could do this with this tool:

In [ ]:
!fastq_to_fasta -h

Then an example could be:

In [ ]:
vsearch --cluster_fast your_file.fasta --id 1.00 --centroids your_file_centroids.fasta --uc your_file.uc